In [9]:
import pandas as pd
import numpy as np
import random

In [10]:
def fitness(pop,trainset,label):
    fitn=[]
    for i in range(len(pop)):
        fit=(pop[i]*trainset).apply(lambda x:x.sum(),axis=1)
        result=0
        for value in range(len(fit)):
            if (fit[value]<0 and label[value]==0) or (fit[value]>=0 and label[value]==1):
                result=result+1
        fitn.append(result/len(fit))
    return fitn

In [11]:
def selection(pop,adap,k):#binary tournament selection
    totalAdap=sum(adap)
    popToChild=[]
    for i in range(int(len(pop)*k)):
        group={}
        for i in range(2):
            target=int(random.random()*len(pop))
            group[adap[target]]=pop[target]
        popToChild.append(group[max(group)])
    return popToChild

In [30]:
def crossOver(pop,adap,children):
    while len(pop)>len(children):
        #use binary tournament selection to get parents.
        father=selection(pop,adap,1/len(pop))
        mother=selection(pop,adap,1/len(pop))
        crossPoint=random.randint(0,len(pop[0])-1)
        children.append(mother[0][:crossPoint]+father[0][crossPoint:])
        children.append(father[0][:crossPoint]+mother[0][crossPoint:])
    return children

In [31]:
def mutation(pop,pm):
    for i in range(len(pop)):
        if random.random()<pm:
            mPoint=random.randint(0,len(pop[0])-1)
            pop[i][mPoint]=random.uniform(-1.0,1.0)
    return pop

In [42]:
if __name__ == "__main__":
    popSize=1000
    chromLength=10
    pc=0.6
    pm=0.01
    population=np.random.uniform(low=-1.0,high=1.0,size=(popSize,chromLength)).tolist()
    
    #read all training instances and split the expected output from them
    total=pd.read_csv('data/training-set.csv',header=None)
    trainset=total.drop(9,axis=1)
    #add one to add one more input whose value is always 1
    trainset[9]=1
    label=pd.DataFrame(total[9])
    label.columns=['label']
    
    fittest=[0,0,[]]
    
    for i in range(200):
        adaptability=fitness(population,trainset,label.values)
        #print(max(adaptability))
        if max(adaptability)>fittest[1]:
            fittest[0]=i+1
            fittest[1]=max(adaptability)
            fittest[2]=population[adaptability.index(max(adaptability))]
        if np.mean(adaptability)>0.9:
            break
        #print(np.mean(adaptability))
        #print(fittest)
        children=selection(population,adaptability,0.1)
        children=crossOver(population,adaptability,children)
        population=mutation(children,pm)
    print(fittest)

[10, 1.0, [0.10835625730910392, -0.26118720522875427, 0.5975135937980662, -0.39733828505553825, 0.9835589946874144, 0.15326222463671235, -0.6864077409339824, -0.5816897543194925, 0.3122516509516051, -0.07787506828291235]]
